In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch

In [2]:
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'

In [3]:
data = pd.read_csv('../input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict,
                       index_col = 0, chunksize = 3141410) #3141410

In [4]:
data = next(iter(data))

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
target = data['target']
data.drop(['target', 'investment_id'], inplace=True, axis=1)

In [6]:
#from scipy.stats import pearsonr 
#def est_score(model):
 #   y_pred = model.predict(X_test)
 #   return pearsonr(y_pred, y_test)[0]

In [7]:
#from sklearn.model_selection import TimeSeriesSplit

In [8]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, stratify=data.index)

In [9]:
#del data, target

In [10]:
#from sklearn.linear_model import LinearRegression
#reg_model = LinearRegression(copy_X=False).fit(X_train, y_train)

In [11]:
#est_score(reg_model)

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_to(data, device):
    """
    moving data to device
    :param data: data to move
    :param device: device
    :return: moved data
    """
    if isinstance(data, (list, tuple)):
        return [move_to(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [13]:
class DNN_Model(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear_3 = torch.nn.Linear(deep_size, deep_size)
        self.linear_4 = torch.nn.Linear(deep_size, deep_size)
        self.linear_5 = torch.nn.Linear(deep_size, int(deep_size/3.5))
        self.linear_6 = torch.nn.Linear(int(deep_size/3.5), 1)
        self.dout = torch.nn.Dropout(p=0.7)
        self.act = torch.nn.SiLU()
    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        self = move_to(self, device)
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        
        if subset == 1:
            shift_subset = 0
        else:
            shift_subset = np.random.randint(0, y.shape[0]-int(y.shape[0]*subset))
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]:   
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[shift_subset+i:shift_subset+i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[shift_subset+i:shift_subset+i+batch_size].values), device)
                loss = torch.nn.MSELoss()(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = torch.nn.MSELoss()(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_5(tensor_X))
        tensor_X = self.linear_6(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        tensor_X = move_to(tensor_X, device)
        tensor_X = self.act(self.linear_1(tensor_X))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_5(tensor_X))
        tensor_X = self.linear_6(tensor_X)
        return tensor_X

In [14]:
class DNN_Model_1(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        half_deep = int(deep_size/2)
        quot_deep = int(half_deep/2)
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear_3 = torch.nn.Linear(deep_size, 1)
        
        self.small_dout = torch.nn.Dropout(p=0.35)
        self.dout = torch.nn.Dropout(p=0.7)
        self.act = torch.nn.SiLU()
        
    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        self = move_to(self, device)
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        
        if subset == 1:
            shift_subset = 0
        else:
            shift_subset = np.random.randint(0, y.shape[0]-int(y.shape[0]*subset))
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]:
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[shift_subset+i:shift_subset+i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[shift_subset+i:shift_subset+i+batch_size].values), device)
                loss = torch.nn.MSELoss()(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = torch.nn.MSELoss()(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor.view(X_tensor.shape[0], 1, -1), device)
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.act(self.linear_3(tensor_X))
        return tensor_X
    

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        tensor_X = move_to(tensor_X.view(tensor_X.shape[0], 1, -1), device)
        tensor_X = self.act(self.linear_1(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_2(tensor_X))
        
        tensor_X = self.act(self.linear_3(tensor_X))
        return tensor_X

In [15]:
class StackedDNN:
    def __init__(self, n, model=None, min_deep=30, max_deep=40):
        if model == None:
            raise Exception('No model selected')
        self.nets = [model(np.random.randint(min_deep, max_deep)) for i in range(n)]
    
    def fit(self, X, y, min_iter=10, max_iter=15, batch_size=4096, subset=1):
        for i in range(len(self.nets)):
            self.nets[i].fit(X, y, np.random.randint(min_iter, max_iter), batch_size, subset, silent=True)
            print(f'{i}/{len(self.nets)-1} fitted')
        
    def predict(self, X):
        y_pred = np.ndarray([len(self.nets), X.shape[0]])
        for i in range(len(self.nets)):
            self.nets[i].eval()
            y_pred[i] = np.array(self.nets[i].forward(X).detach().view(-1).cpu())
        y_pred /= len(self.nets)
        return np.sum(y_pred, axis=0)

In [16]:
nn_model = StackedDNN(10, model=DNN_Model, min_deep=289, max_deep=300)
nn_model.fit(data, target, min_iter=20, max_iter=30, batch_size=4096*10)

#nn_model = StackedDNN(1, model=DNN_Model, min_deep=100, max_deep=120)
#nn_model.fit(X_train, y_train, min_iter=35, max_iter=38, batch_size=4096*10)


0/9 fitted
1/9 fitted
2/9 fitted
3/9 fitted
4/9 fitted
5/9 fitted
6/9 fitted
7/9 fitted
8/9 fitted
9/9 fitted


In [17]:
#res = nn_model.predict(X_test)
#print(pearsonr(res, y_test)[0])

In [18]:
nn_model_1 = StackedDNN(10, model=DNN_Model_1, min_deep=289, max_deep=300)
nn_model_1.fit(data, target, min_iter=20, max_iter=22, batch_size=4096*5)

0/9 fitted
1/9 fitted
2/9 fitted
3/9 fitted
4/9 fitted
5/9 fitted
6/9 fitted
7/9 fitted
8/9 fitted
9/9 fitted


In [19]:
#res_1 = nn_model_1.predict(X_test)
#print(pearsonr(res_1, y_test)[0])

In [20]:
#res = nn_model.predict(X_test)
#res_1 = nn_model_1.predict(X_test)

In [21]:
#a = 0.5

#print(pearsonr(res, y_test)[0])
#print(pearsonr(res_1, y_test)[0])
#print(pearsonr(a*res + (1-a)*res_1, y_test)[0])
#0.185525751867734 single
#0.18855076782948865 -2
#0.1925553136005383 -5
#0.1904118171465486 -10

#несмотря на то что model_1 явно хуже model, результат композиции превосходит их по отдельности
#написать несколько сетей разной архитектуры, и улучшить предыдушие две

#0.18803793483533365 res -1
#0.16739222414637583 res_1 -1
#0.19103148334509518 composition -1

In [22]:
#res_2 = (res+res_1)/2

##### 

In [23]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=10e3, task_type='GPU', devices='0', silent=True).fit(data, target)

In [24]:
#est_score(model)

In [25]:
#boost_pred = model.predict(X_test)

In [26]:
#a = 0.7
#print(pearsonr(a*boost_pred + (1-a)*res_2, y_test)[0])

In [27]:
#res_boost = model.predict(X_test)

In [28]:
#a = 0.5
#pearsonr(a*res_boost+(1-a)*res, y_test)[0]

In [29]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    res_boost = model.predict(test_df[features])
    res_nn = nn_model.predict(test_df[features])
    res_nn_1 = nn_model_1.predict(test_df[features])
    test_df['target']  = 0.7*res_boost + 0.3*(res_nn + res_nn_1)/2
    env.predict(test_df[['row_id','target']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [30]:
.902

0.902